# Lab 6: Text Embedding

The goal of this lab is to study text embedding. To do this, we will follow several steps:
1) Tokenization
2) Token IDs (integer identifiers)
3) Special tokens
4) A famous tokenizer: Byte-Pair Encoding (BPE)
5) Data sampling
6) Token Embedding
7) Positional embedding.

Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

import os
import urllib.request

import re
# This module provides regular expression matching operations similar to those found in Perl.

import importlib
import tiktoken
# tiktoken is a fast tokeniser for use with OpenAI's models.
print("tiktoken version:", importlib.metadata.version("tiktoken"))

import torch
print("PyTorch version:", torch.__version__)

from torch.utils.data import Dataset, DataLoader


## 1- Breaking text into small units

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters
- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [ ]:
file_path = "the-verdict.txt"
if not os.path.exists(file_path):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    urllib.request.urlretrieve(url, file_path)

Read the text file

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

- The goal is to tokenize and embed this text for an LLM
- The following regular expression will split on whitespaces. It is tested on a small text.

In [ ]:
text = "Hello, world. Is this-- a test?"

# We don't only want to split on whitespaces but also commas and periods, so let's modify the regular expression to do that as well
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

# Strip whitespace from each item and then filter out any empty strings.
# The strip() method removes any leading, and trailing whitespaces.
result = [item.strip() for item in result if item.strip()]
print(result)

- This is pretty good, and we are now ready to apply this tokenization to the raw text

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed)) # total number of tokens

## 2- Converting tokens into token IDs

- From these tokens, we can now build a vocabulary that consists of all the unique tokens

In [ ]:
# set: remove the duplicates
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

- Next, we convert the text tokens into token IDs that we can process via embedding layers later

In [ ]:
# Define the vocabulary as the dictionnary of tokens
vocab = {token:integer for integer,token in enumerate(all_words)}



### Question 1: give the Token ID of the token "now"?

In [ ]:
# Write down your code here.

### Question 2: what is the role of "encode" in the following class? What is the role of "decode"?

Write down your answer here.

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

# We can decode the integers back into text
print(tokenizer.decode(ids))

## 3- Adding special context tokens

**General "special" tokens**

- It's useful to add some "special" tokens for unknown words and to denote the end of a text
- Some tokenizers use special tokens to help the LLM with additional context
- Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks the beginning of text
  - `[EOS]` (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
  - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
- `[UNK]` to represent words that are not included in the vocabulary




**GPT-2 "special" tokens**

- Note that GPT-2 does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token to reduce complexity
- The `<|endoftext|>` is analogous to the `[EOS]` token mentioned above
- GPT also uses the `<|endoftext|>` for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)
- GPT-2 does not use an `<UNK>` token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units
- We use the `<|endoftext|>` tokens between two independent sources of text


### Question 3: why does an error occur in the following cell?

Write down your answer here.

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

#tokenizer.encode(text)

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

### Question 4: modify the class "Simple TokenizerV1" to use the `<unk>` token when the token is not in the predefined vocabulary. The modified class will be named "SimpleTokenizerV2".

In [ ]:
# Write down your code here.

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        pass
        return ids

    def decode(self, ids):
        pass
        return text

Let's try to tokenize text with the modified tokenizer:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

print(tokenizer.encode(text))

print(tokenizer.decode(tokenizer.encode(text)))

## 4- Byte-Pair encoding

- GPT-2 used Byte-Pair encoding (BPE) as its tokenizer
- It allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- We are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

strings = tokenizer.decode(integers)

print(strings)

## 5- Data sampling with a sliding window

- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict.
- In the following sentence for instance, we want to predict `notebook` from the words occuring before `notebook`. We don't use the words occuring after it. The LLM can't acces words past the target.
- Example : The students opened their notebook when the lecture begins.



In [ ]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

- For each text chunk, we want the inputs and targets
- Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

### Question 5: what is the meaning of "context_size" in the following cell?

Write your answer here

In [ ]:
context_size = 4

enc_sample = enc_text[50:]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

- One by one, the prediction would look like as follows:

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]),"\n")

- Create dataset and dataloader that extract chunks from the input text dataset
- The dataloader iterates over the input dataset and returns the inputs and targets shifted by one

In [ ]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

### Question 6: test the data loader "create_dataloader_v1" with a batch size of 1, a context size of 4 and a stride of 1. Be sure to understand the content of a batch.

In [ ]:
# Write down your code here

- We can also create batched outputs
- Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

## 6- Creating token embeddings

- The data is already almost ready for an LLM
- But lastly let us embed the tokens in a continuous vector representation using an embedding layer
- Usually, these embedding layers are part of the LLM itself and are updated (trained) during model training
- Suppose we have the following four input examples with input ids 2, 3, 5, and 1 (after tokenization):

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])

- For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want to create embeddings of size 3

### Question 7: what is the role of "torch.nn.Embedding" in the following cell? How is it initialized?

Write your answer here.

In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(4598)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print(embedding_layer.weight)

- To convert a token with id 5 into a 3-dimensional vector, we do the following:

In [ ]:
print(embedding_layer(torch.tensor([5])))

## 7- Encoding word positions

- Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence
- Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model

 ### Question 8: create an embedding, named "token_embedding_layer", for the BytePair encoder into a 256-dimensional vector representation.

Reminder: The BytePair encoder has a vocabulary size of 50,257

In [ ]:
# Write down your code here

 ### Question 9: use "create_dataloader_v1" to generate a batch of size 8 and a context size of 4. Embed the tokens of this batch with "token_embedding_layer".

In [ ]:
max_length = 4
# Write down your code here

- GPT-2 uses absolute position embeddings, so we just create another embedding layer

 ### Question 10: create an embedding layer "pos_embedding_layer" for position embeddings. Choose correctly the parameters of this embedding.

In [ ]:
# Write down your code here

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

print(pos_embeddings)

- To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
